In [1]:
!apt-get install llvm-9-dev
!pip install cpufeature
!pip install triton==0.2.3
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
!pip install transformers
!pip install git+https://github.com/RussianNLP/russian_paraphrasers@master

  Cloning https://github.com/RussianNLP/russian_paraphrasers (to revision master) to /tmp/pip-req-build-xnjmva5h
  Running command git clone -q https://github.com/RussianNLP/russian_paraphrasers /tmp/pip-req-build-xnjmva5h
  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-install-t4mnirr8/nlg-eval_be7b67cee49d45e1b5cb93af88d8e202
  Running command git clone -q https://github.com/Maluuba/nlg-eval.git /tmp/pip-install-t4mnirr8/nlg-eval_be7b67cee49d45e1b5cb93af88d8e202
  Using cached sentence_transformers-0.4.0-py3-none-any.whl


  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 1.2.0
    Uninstalling sentence-transformers-1.2.0:
      Successfully uninstalled sentence-transformers-1.2.0


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [5]:
from src.xl_wrapper import RuGPT3XL

In [6]:
gpt = RuGPT3XL.from_pretrained(
    "sberbank-ai/rugpt3xl",
    weights_path="./model/mp_rank_00_model_states.pt",
    deepspeed_config_path="./src/deepspeed_config/gpt3_xl_sparse_2048.json",
    seq_len=512
)

> initializing model parallel with size 1
Use alternating sparse & dense attention layers
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234


In [7]:
gpt.tokenizer.add_special_tokens({"bos_token": "<s>"})
gpt.tokenizer.add_special_tokens({"eos_token": "</s>"})
gpt.tokenizer.add_special_tokens({"sep_token": "==>"})

1

In [8]:
def generate(text, additional_len=150):
    return filter_resuls(gpt.generate(
        text=text,
        max_length=len(gpt.tokenizer.encode(text)) + additional_len,
        eos_token_id=gpt.tokenizer.eos_token_id,
        do_sample=True,
        num_return_sequences=5,
        top_k=10,
        top_p=0.9
    ))

def generate_beam(text, additional_len=150):
    return filter_resuls(gpt.generate(
        text=text,
        max_length=len(gpt.tokenizer.encode(text)) + additional_len,
        eos_token_id=gpt.tokenizer.eos_token_id,
        num_beams=5,
        num_return_sequences=1
    ))

def generate_nosample(text, additional_len=200):
    return filter_resuls(gpt.generate(
        text=text,
        max_length=len(gpt.tokenizer.encode(text)) + additional_len,
        eos_token_id=gpt.tokenizer.eos_token_id,
        do_sample=False
    ))

def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

In [9]:
from russian_paraphrasers import GPTParaphraser
import difflib
import re

paraphraser = GPTParaphraser(model_name="gpt3", range_cand=True, make_eval=False)


def check_bad_generations(origin, pred):
    bad = False
    origin = origin.strip()
    pred = pred.strip()
    ratio = difflib.SequenceMatcher(None, origin.lower(), pred.lower()).ratio()
    if ratio > 0.97:
#         print("REPEAT!")
#         print(origin, "===> ", pred.strip())
        bad  = True

    # Length
    if len(origin) - len(pred) > len(origin) - 50:
#         print("Length: ", origin, "===> ", pred)
        bad = True
        
    # check separation union
    origin_t = re.sub("[^а-я ]", "", origin.lower())
    or_tokens = set(origin_t.split())

    pred_t = re.sub("[^а-я ]", "", pred.lower())
    pr_tokens = set(pred_t.split())

#   Maybe without this part!
    sep = (pr_tokens & or_tokens)
    if len(sep) < 3:
        # print("Smth strange:", pr_tokens, or_tokens)
#         print("or: ", origin)
#         print("===> ", pred)
        bad = True
    return bad

def get_paraphraser_var(origin):
    results = paraphraser.generate(
            origin, n=10, temperature=1, 
            top_k=10, top_p=0.9, 
            max_length=200, repetition_penalty=1.5,
            threshold=0.8
        )
    try:
        res = results["results"][0]["best_candidates"][0]
        if res.endswith("?"):
            res = results["results"][0]["best_candidates"][1]
    except IndexError:
            res = results["results"][0]["predictions"][0]
    return res

PyTorch version 1.6.0+cu101 available.
TensorFlow version 2.3.0 available.


In [10]:
import csv
import random
import pprint
import pandas as pd
import tqdm

to_model = []

with open("./hidden_test_only.csv") as csv_file: #public_test_only
    for row in csv_file.readlines():
        source = row.strip()
        to_model.append(source)

print(len(to_model))

with open("hypothesis_gptXL_private_subset_check_no_sample.txt", "w") as out_file:
    for sentence in tqdm.tqdm(to_model):
        mysentence = "<s>" + sentence + " ==> "
        
        ress = generate_nosample(sentence)[0]
        try:
            result = ress.split("==>")[1].replace("</s>", "").replace("</s", "").replace("<pad>", "").replace("<pad", "").strip()
            if check_bad_generations(sentence, result):
                result = get_paraphraser_var(sentence)
            out_file.write(result+"\n")
        except:
            result = get_paraphraser_var(sentence)
            out_file.write(result+"\n")

  0%|          | 0/1126 [00:00<?, ?it/s]

1126


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 12%|█▏        | 138/1126 [05:58<57:34,  3.50s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 14%|█▎        | 153/1126 [06:32<14:41,  1.10it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 21%|██        | 236/1126 [09:31<25:41,  1.73s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 25%|██▌       | 287/1126 [11:46<20:41,  1.48s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 39%|███▉      | 438/1126 [17:32<20:55,  1.83s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 46%|████▌     | 519/1126 [20:46<31:15,  3.09s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 49%|████▊     | 547/1126 [22:00<17:17,  1.79s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 49%|████▉     | 550/1126 [22:16<38:15,  3.98s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 52%|█████▏    | 584/1126 [23:34<21:22,  2.37s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 63%|██████▎   | 713/1126 [27:52<18:25,  2.68s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 67%|██████▋   | 753/1126 [29:22<08:01,  1.29s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 80%|███████▉  | 898/1126 [35:04<05:15,  1.38s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 84%|████████▎ | 942/1126 [36:47<08:50,  2.88s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 94%|█████████▍| 1059/1126 [40:49<01:43,  1.54s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|██████████| 1126/1126 [43:57<00:00,  2.34s/it]


### To check of two hypothessis use Sentence Transformer

In [11]:
!pip install -U sentence-transformers


from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

  Using cached sentence_transformers-1.2.0-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 0.4.0
    Uninstalling sentence-transformers-0.4.0:
      Successfully uninstalled sentence-transformers-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
russian-paraphrasers 0.0.3 requires sentence-transformers==0.4.0, but you have sentence-transformers 1.2.0 which is incompatible.


In [12]:
public_test = ['Facebook по инициативе Роскомнадзора неоднократно привлекалась к административной ответственности постановлениями мировых судей судебных участков района "Таганский" города Москвы.']
data_preds_1 = ["Роскомнадзор привлекался к ответственности за нарушение Facebook авторских прав."]
data_preds_2 = ["Роскомнадзор неоднократно привлекал Facebook к административной ответственности по постановлению мирового судьи."]


best_pred = []
first = 0
for origin, pr1, pr2 in zip(public_test, data_preds_1, data_preds_2):

    or_embedding = model.encode(origin)
    pr1_embedding = model.encode(pr1)
    pr2_embedding = model.encode(pr2)

    op1 = util.pytorch_cos_sim(or_embedding, pr1_embedding)
    op2 = util.pytorch_cos_sim(or_embedding, pr2_embedding)
    print(op1, op2)
    
    if op1 > op2:
        best_pred.append(pr1)
        first += 1
    else:
        best_pred.append(pr2)

print(best_pred)

tensor([[0.6543]]) tensor([[0.8849]])
['Роскомнадзор неоднократно привлекал Facebook к административной ответственности по постановлению мирового судьи.']
